# 14 Um Modelo Hierárquico

## 14.1 - O problema do contador Geiger

![](Geiger.png)

- Serve para medir certas radiações ionizantes:
    - Partículas alfa, beta ou radiação gama;
    - Raios-X.

> ---
> Suponha que uma fonte radioativa emita partículas em direção a um contador Geiger a uma taxa média de r partículas por segundo, mas o contador registra apenas uma fração, f , das partículas que o atingem. Se f é 10% e o contador registra 15 partículas em um intervalo de um segundo, qual é a distribuição posterior de n , o número real de partículas que atingem o contador e r, a taxa média de partículas emitidas?> 

Cadeia de causalidade que começa com os parâmetros do sistema e termina com os dados observados:
1. A fonte emite partículas a uma taxa média, r .
2. Durante um determinado segundo, a fonte emite n partículas em direção ao contador.
3. Dessas n partículas, algum número, k, é contado.

- O decaimento radioativo é bem modelado por um processo de Poisson;
    - Dado r, a distribuição de n é a distribuição de Poisson com o parâmetro r.
- A probabilidade de detecção para cada partícula é independente dos outros;
    - A distribuição de k é a distribuição binomial com parâmetros n e f.

**Problema avançado**: Dados os parâmetros do sistema, queremos a distribuição dos dados;

**Problema inverso**: Dados os dados, queremos a distribuição dos parâmetros;
    
- E se você puder resolver o **problema avançado**, poderá usar métodos bayesianos para resolver o **problema inverso**.

## 14.2 - Comece simples

![](post_14_2.png)

Distribuição posteriori de n para vários valores dados de r.

- Sabemos o valor de r;
- Nos é dado o valor de f;
- Precisamos fazer é estimar n .

In [1]:
import thinkbayes
class Detector(thinkbayes.Suite):

    def __init__(self, r, f, high=500, step=1):
        pmf = thinkbayes.MakePoissonPmf(r, high, step=step)
        thinkbayes.Suite.__init__(self, pmf, name=r)
        self.r = r
        self.f = f
    
    def Likelihood(self, data, hypo):
        k = data
        n = hypo
        p = self.f

        return thinkbayes.EvalBinomialPmf(k, n, p)

In [2]:
f = 0.1
k = 15

for r in [100, 250, 400]:
    suite = Detector(r, f, step=1)
    suite.Update(k)
    print(suite.MaximumLikelihood())

104
239
375


A probabilidade de detectar k partículas é dada pela distribuição binomial.

## 14.3 - Torne-o hierárquico

- Na seção anterior, assumimos que r é conhecido;

In [9]:
class Emitter(thinkbayes.Suite):

    def __init__(self, rs, f=0.1):
        detectors = [Detector(r, f) for r in rs]
        thinkbayes.Suite.__init__(self, detectors)
        
    def Likelihood(self, data, hypo):
        return hypo.Update(data)
    
    def Update(self, data):
        thinkbayes.Suite.Update(self, data)
        
        for detector in self.Values():
            detector.Update()

Modela o comportamento do emissor e estima r;

Emitter é um **meta-Suite**; que é, uma Suite que contém outros valores de Suite.

Um modelo com vários níveis de Suites, é chamado **hierárquico**;

## 14.4 - Um pouco de otimização

Otimiza a verossimilhança da clase Emitter;

## 14.5 - Extraindo as posteriores

![](post_14_5.png)

Distribuições posteriores de n e r;

In [10]:
class Emitter(thinkbayes.Suite):

    def __init__(self, rs, f=0.1):
        detectors = [Detector(r, f) for r in rs]
        thinkbayes.Suite.__init__(self, detectors)
        
    def Likelihood(self, data, hypo):
        return hypo.Update(data)
    
    def Update(self, data):
        thinkbayes.Suite.Update(self, data)
        
        for detector in self.Values():
            detector.Update()
    
    def DistOfR(self):
        items = [(detector.r, prob) for detector, prob in self.Items()]
        return thinkbayes.MakePmfFromItems(items)
    
    def DistOfN(self):
        return thinkbayes.MakeMixture(self)

- Podemos obter a distribuição posterior de r percorrendo os detectores e suas probabilidades;
- A distribuição posterior de n, é a mistura dos detectores (DistOfN);
- As distribuições posteriores de R e n são semelhantes;

## 14.6 - Discussão

A taxa de emissão r tem um efeito causal no número de partículas, n, que tem um efeito causal na contagem de partículas, k.

**Modelo hierárquico**: Causas na parte superior e efeitos na parte inferior.

1. No nível superior, intervalo de valores hipotéticos para r.
2. Para cada valor de r, temos um intervalo de valores para n, e a distribuição a priori de n depende de r.
3. Quando atualizamos o modelo, vamos de baixo para cima. 
    - Calculamos uma distribuição posteriori de n para cada valor de r e depois calculamos a distribuição posteriori de r.